In [1]:
using Pkg;
Pkg.activate("..");
Pkg.instantiate();

  Activating project at `~/dev/uni/amo-team-project`


In [2]:
using HiGHS
using JuMP
using Ipopt
using DataFrames
using JSON
using Plots
using StatsPlots
using StatsBase
using BenchmarkTools
using Distributions
using Distances

InitError: InitError: could not load library "/home/moritz/.julia/artifacts/cdf6dc8aa6771a61c6c65a5a5c1a8d1b75f50a2f/lib/libarpack.so"
libopenblas64_.so: cannot open shared object file: No such file or directory
during initialization of module Arpack_jll

In [105]:
include("../src/rwth_parser.jl")
network, loads = read_data("../data/Scenario_2013.xlsx")
print()

In [4]:
df = read_sheet("../data/Scenario_2013.xlsx", LOAD_SHEET_NAME)
print()

In [5]:
include("../src/loads.jl")
local_loads = remove_non_data_rows(filter_month(loads, 1))
print()

In [138]:
include("../src/scenarios.jl")
attack_scenarios = generate_attack_scenarios(network, 10, 1000)
weather_scenarios = generate_weather_scenarios(1000)
print()

In [141]:
reduced_attack_scenarios = reduce_binary_scenarios(attack_scenarios, 10)
reduced_weather_scenarios = reduce_continous_scenarios(weather_scenarios, 10)
reduced_load_scenarios = reduce_continous_scenarios(local_loads, 10)

scenarios = cartesian_scenarios(
	reduced_attack_scenarios,
	reduced_weather_scenarios,
	reduced_load_scenarios,
)

print()

In [97]:
include("../src/model.jl")
dispatch_model = build_model(network, scenarios, budget)
print()

In [98]:
solve!(dispatch_model)

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
467 rows, 467 cols, 1359 nonzeros
392 rows, 392 cols, 1289 nonzeros
273 rows, 296 cols, 1019 nonzeros
231 rows, 270 cols, 934 nonzeros

Solving MIP model with:
   231 rows
   270 cols (7 binary, 0 integer, 0 implied int., 263 continuous)
   934 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   37.68           inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   157.647185      236.8231947       33.43%        0      0      0       154     0.0s
 C       0       0         0   0.00%   165.8077366     236.8231947       29.99%      140     12      0       183     0.0s
 L  

OPTIMAL::TerminationStatusCode = 1

In [89]:
results_r = convert_jump_container_to_df(vars[:r])

Row,dim1,Value
,Symbol,Float64
1,B1,0.0
2,B10,0.0
3,B11,0.0
4,B12,0.0
5,B13,0.0
6,B14,0.0
7,B2,0.0
8,B3,1.0
9,B4,0.0


In [92]:
p_results = reformat_2d_df(convert_jump_container_to_df(vars[:P]))

Row,dim1,S1,S10,S2,S3,S4,S5,S6,S7,S8,S9
,Symbol,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,G1,147.424,1.01601e-14,-0.0,27.721,144.671,158.101,27.721,27.721,147.424,27.721
2,G2,40.0,0.0,-0.0,0.0,35.6112,40.0,-5.04522e-16,-1.02002e-15,40.0,1.66247e-16


In [67]:
m = JuMP.Model(HiGHS.Optimizer)

# variables
## first-stage
@variable(m, r[b in keys(busses)], Bin) # reinforce bus

## second-stage
@variable(m, 0 <= P[g in keys(generators), s in keys(scenarios)]) # power generation
@variable(m, 0 <= L_shed[b in keys(busses), s in keys(scenarios)] <= busses[b][:load]) # load shedding

@variable(m, z[b in keys(busses), s in keys(scenarios)], Bin) # bus outage

@variable(m, F[l in keys(lines), s in keys(scenarios)]) # power flow
@variable(m, δ[b in keys(busses), s in keys(scenarios)]) # voltage angle

print()

In [158]:
# objective
# minimize expected load shedding over all scenarios
@objective(m, Min, sum(sum(L_shed[b, s] for b in keys(busses)) * 1/length(scenarios) for s in keys(scenarios)))
print()

In [159]:
# constraints

## first-stage
### reinforcement budget
@constraint(m, 
	reinforcement_budget,
	  sum(r[b] * busses[b][:reinforcement_cost] for b in keys(busses)) 
	<= budget
)

## second-stage
### power balance
@constraint(m, 
	power_balance[b in keys(busses), s in keys(scenarios)],
	sum(P[g, s] for g in busses[b][:generators]) 
	+ sum(F[l, s] for l in busses[b][:incoming]) 
	- sum(F[l, s] for l in busses[b][:outgoing]) 
	+ L_shed[b, s] 
	- busses[b][:load]
	== 0
)
### line flow
@constraint(m, 
	line_flow[l in keys(lines), s in keys(scenarios)],
	F[l, s] == lines[l][:susceptance] * (δ[lines[l][:from], s] - δ[lines[l][:to], s])
)
### line flow under outage upper
@constraint(m,
	line_flow_under_outage_upper[b in keys(busses), l in [busses[b][:incoming]; busses[b][:outgoing]], s in keys(scenarios)],
	F[l, s] <= lines[l][:capacity] * z[b, s]
)
### line flow under outage lower
@constraint(m,
	line_flow_under_outage_lower[b in keys(busses), l in [busses[b][:incoming]; busses[b][:outgoing]], s in keys(scenarios)],
	-lines[l][:capacity] * z[b, s] <= F[l, s] 
)

### generator under outage lower
@constraint(m,
	generator_under_outage_lower[g in keys(generators), s in keys(scenarios)],
	-generators[g][:capacity] * z[generators[g][:bus], s] <= P[g, s] 
)
### generator under outage upper
@constraint(m,
	generator_under_outage_upper[g in keys(generators), s in keys(scenarios)],
	P[g, s] <= generators[g][:capacity] * z[generators[g][:bus], s]
)

### reference angle
@constraint(m,
	reference_angle[s in keys(scenarios)],
	δ[:B1, s] == 0
)
### attacked busses
@constraint(m,
	busses[s in keys(scenarios), b in keys(busses)],
	z[b, s] <= 1 - (b in scenarios[s][:attacked_busses] ? 1 : 0) + r[b]
)

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:S4, :S2, :S1, :S3]
    Dimension 2, [:B3, :B1, :B2]
And data, a 4×3 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 busses[S4,B3] : -r[B3] + z[B3,S4] ≤ 1.0  …  busses[S4,B2] : -r[B2] + z[B2,S4] ≤ 0.0
 busses[S2,B3] : -r[B3] + z[B3,S2] ≤ 1.0     busses[S2,B2] : -r[B2] + z[B2,S2] ≤ 1.0
 busses[S1,B3] : -r[B3] + z[B3,S1] ≤ 1.0     busses[S1,B2] : -r[B2] + z[B2,S1] ≤ 1.0
 busses[S3,B3] : -r[B3] + z[B3,S3] ≤ 1.0     busses[S3,B2] : -r[B2] + z[B2,S3] ≤ 0.0

In [160]:
print(m)

Min 0.25 L_shed[B3,S4] + 0.25 L_shed[B1,S4] + 0.25 L_shed[B2,S4] + 0.25 L_shed[B3,S2] + 0.25 L_shed[B1,S2] + 0.25 L_shed[B2,S2] + 0.25 L_shed[B3,S1] + 0.25 L_shed[B1,S1] + 0.25 L_shed[B2,S1] + 0.25 L_shed[B3,S3] + 0.25 L_shed[B1,S3] + 0.25 L_shed[B2,S3]
Subject to
 power_balance[B3,S4] : L_shed[B3,S4] + F[L2,S4] - F[L3,S4] = 200.0
 power_balance[B1,S4] : P[G1,S4] + L_shed[B1,S4] - F[L2,S4] + F[L1,S4] = 0.0
 power_balance[B2,S4] : P[G2,S4] + L_shed[B2,S4] - F[L1,S4] + F[L3,S4] = 0.0
 power_balance[B3,S2] : L_shed[B3,S2] + F[L2,S2] - F[L3,S2] = 200.0
 power_balance[B1,S2] : P[G1,S2] + L_shed[B1,S2] - F[L2,S2] + F[L1,S2] = 0.0
 power_balance[B2,S2] : P[G2,S2] + L_shed[B2,S2] - F[L1,S2] + F[L3,S2] = 0.0
 power_balance[B3,S1] : L_shed[B3,S1] + F[L2,S1] - F[L3,S1] = 200.0
 power_balance[B1,S1] : P[G1,S1] + L_shed[B1,S1] - F[L2,S1] + F[L1,S1] = 0.0
 power_balance[B2,S1] : P[G2,S1] + L_shed[B2,S1] - F[L1,S1] + F[L3,S1] = 0.0
 power_balance[B3,S3] : L_shed[B3,S3] + F[L2,S3] - F[L3,S3] = 200.0
 

In [161]:
optimize!(m)

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
49 rows, 38 cols, 114 nonzeros
32 rows, 21 cols, 86 nonzeros
29 rows, 15 cols, 67 nonzeros
17 rows, 14 cols, 47 nonzeros

Solving MIP model with:
   17 rows
   14 cols (1 binary, 0 integer, 0 implied int., 13 continuous)
   47 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   50              inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   75              100               25.00%        0      0      0         9     0.0s
 C       0       0         0   0.00%   94.44375921     100                5.56%       48      3      0        16     0.0s

Solving report
  Stat

# Results
## Reinforce results

In [83]:
"""
Returns a `DataFrame` with the values of the variables from the JuMP container `var`.
The column names of the `DataFrame` can be specified for the indexing columns in `dim_names`,
and the name of the data value column by a Symbol `value_col` e.g. :Value
"""
function convert_jump_container_to_df(
        var::JuMP.Containers.DenseAxisArray;
        dim_names::Vector{Symbol}=Vector{Symbol}(),
        value_col::Symbol=:Value
    )

    if isempty(var)
        return DataFrame()
    end

    if length(dim_names) == 0
        dim_names = [Symbol("dim$i") for i in 1:length(var.axes)]
    end

    if length(dim_names) != length(var.axes)
        throw(ArgumentError("Length of given name list does not fit the number of variable dimensions"))
    end

    tup_dim = (dim_names...,)

    # With a product over all axis sets of size M, form an Mx1 Array of all indices to the JuMP container `var`
    ind = reshape([collect(k[i] for i in 1:length(dim_names)) for k in Base.Iterators.product(var.axes...)],:,1)

    var_val  = value.(var)

    df = DataFrame([merge(NamedTuple{tup_dim}(ind[i]), NamedTuple{(value_col,)}(var_val[(ind[i]...,)...])) for i in 1:length(ind)])

    # sort by :dim1

    df = df[sortperm(df[!, Symbol("dim1")]), :]

    return df
end

convert_jump_container_to_df

In [84]:
function reformat_2d_df(df::DataFrame)
	# sort dim 1 and 2
	df = df[sortperm(df[!, :dim1]), :]
	df = df[sortperm(df[!, :dim2]), :]
	return unstack(df, :dim1, :dim2, :Value)
end

reformat_2d_df (generic function with 1 method)

In [164]:
results_r = convert_jump_container_to_df(r)

Row,dim1,Value
,Symbol,Float64
1,B1,0.0
2,B2,1.0
3,B3,0.0


In [165]:
p_results = reformat_2d_df(convert_jump_container_to_df(P))

Row,dim1,S1,S2,S3,S4
,Symbol,Float64?,Float64?,Float64?,Float64?
1,G1,50.0,0.0,100.0,0.0
2,G2,150.0,5.68434e-14,100.0,0.0


In [166]:
L_shed_results = reformat_2d_df(convert_jump_container_to_df(L_shed))

Row,dim1,S1,S2,S3,S4
,Symbol,Float64?,Float64?,Float64?,Float64?
1,B1,0.0,0.0,0.0,0.0
2,B2,0.0,0.0,0.0,0.0
3,B3,0.0,200.0,0.0,200.0


In [167]:
z_results = reformat_2d_df(convert_jump_container_to_df(z))

Row,dim1,S1,S2,S3,S4
,Symbol,Float64?,Float64?,Float64?,Float64?
1,B1,1.0,0.0,1.0,0.0
2,B2,1.0,1.0,1.0,1.0
3,B3,1.0,1.0,1.0,1.0


In [168]:
F_results = reformat_2d_df(convert_jump_container_to_df(F))

Row,dim1,S1,S2,S3,S4
,Symbol,Float64?,Float64?,Float64?,Float64?
1,L1,33.3333,1.77636e-15,0.0,0.0
2,L2,83.3333,3.01981e-14,100.0,0.0
3,L3,-116.667,-5.50671e-14,-100.0,0.0


In [169]:
δ_results = reformat_2d_df(convert_jump_container_to_df(δ))

Row,dim1,S1,S2,S3,S4
,Symbol,Float64?,Float64?,Float64?,Float64?
1,B1,0.0,0.0,0.0,0.0
2,B2,0.0666667,3.55271e-18,0.0,0.0
3,B3,-0.166667,-6.03961e-17,-0.2,0.0


In [99]:
(reformat_2d_df(convert_jump_container_to_df(model)), F_results = reformat_2d_df(convert_jump_container_to_df(F))

ErrorException: syntax: "convert_jump_container_to_df(δ)" is not a valid function argument name around /home/moritz/dev/uni/amo-team-project/notebooks/sketch.ipynb:1